In [1]:
'''
找就位道、计算倒凹、填倒凹
'''

import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/9月下旬项目重开"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())

obj_file_path = "150颗0806新数据/3.obj"
txt_file_path = "多分类预处理0929/预处理结果文档/3.txt"

当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\9月下旬项目重开


In [2]:
# 1.准备数据
import chardet
import numpy as np

def detect_encoding(file_path):
    '''检测文件的encoding方式'''
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        return result['encoding']
        
def load_obj_file(file_path, encoding):
    '''读取obj文件中的点云坐标，输出是list'''
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return vertices, faces

def insert_midpoint_points(vertices, faces, minimum_vertices_number=3000):
    if len(vertices) >= minimum_vertices_number:
        print('原始点云的点数足够多')
        return vertices, faces

    else:
        temp = 0
        while len(vertices) < minimum_vertices_number:
            edge_to_midpoint = {}
            new_points = []
            new_faces = []
            vertex_offset = len(vertices)
    
            for face in faces:
                # Compute midpoints for each edge
                midpoints = []
                for i in range(len(face)):
                    edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
                    if edge not in edge_to_midpoint:
                        midpoint = np.mean([vertices[edge[0]], vertices[edge[1]]], axis=0)
                        edge_to_midpoint[edge] = vertex_offset + len(new_points)
                        new_points.append(midpoint.tolist())
                    midpoints.append(edge_to_midpoint[edge])
                
                # Original vertices
                v0, v1, v2 = face
                # Midpoints
                m0, m1, m2 = midpoints

                # Create four new faces
                new_faces.append([v0, m0, m2])
                new_faces.append([v1, m1, m0])
                new_faces.append([v2, m2, m1])
                new_faces.append([m0, m1, m2])
    
            vertices.extend(new_points)
            faces = new_faces
            temp += 1

        print(f'使用了 {temp} 轮插值来让点云点数满足要求')
        
    return vertices, faces

def load_labeled_point_cloud(file_path):
    '''读取点云数据和标签'''
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4:
                x, y, z, label = map(float, parts)
                data.append([x, y, z])
                labels.append(int(label))
    return np.array(data), np.array(labels)

In [3]:
import open3d as o3d
import numpy as np

# Möller–Trumbore算法检测射线与三角形相交
def ray_triangle_intersection(ray_origin, ray_direction, triangle):
    EPSILON = 1e-6  # 用于避免浮点数误差

    # 确保 triangle 是三维的
    vertex0, vertex1, vertex2 = triangle
    
    # 计算三角形的两个边
    edge1 = vertex1 - vertex0
    edge2 = vertex2 - vertex0

    # 计算叉积
    h = np.cross(ray_direction, edge2)
    a = np.dot(edge1, h)

    if -EPSILON < a < EPSILON:
        return False, None  # 射线与三角形平行

    f = 1.0 / a
    s = ray_origin - vertex0
    u = f * np.dot(s, h)

    if u < 0.0 or u > 1.0:
        return False, None

    q = np.cross(s, edge1)
    v = f * np.dot(ray_direction, q)

    if v < 0.0 or u + v > 1.0:
        return False, None

    # 计算 t 以确定交点位置
    t = f * np.dot(edge2, q)

    if t > EPSILON:
        intersection_point = ray_origin + ray_direction * t
        return True, intersection_point

    return False, None  # 射线在三角形平面上但没有相交


# 计算三角形的XZ平面投影
def project_triangle_to_plane(triangle, plane="xz"):
    if plane == "xz":
        return triangle[:, [0, 2]]  # 丢弃Y轴
    elif plane == "xy":
        return triangle[:, [0, 1]]  # 丢弃Z轴
    elif plane == "yz":
        return triangle[:, [1, 2]]  # 丢弃X轴

# 判断两个三角形的投影是否重叠（简单AABB判断）
def check_projection_overlap(proj1, proj2):
    min1, max1 = np.min(proj1, axis=0), np.max(proj1, axis=0)
    min2, max2 = np.min(proj2, axis=0), np.max(proj2, axis=0)
    
    # 如果投影的AABB不相交，则没有重叠
    return not (max1[0] < min2[0] or max2[0] < min1[0] or max1[1] < min2[1] or max2[1] < min1[1])

# 对所有三角形进行投影并检测重叠，返回重叠三角形的索引
def find_overlapping_triangles(points, triangles, ray_direction):
    overlapping_triangles = []
    
    # 如果射线方向向上，投影到 XZ 平面
    plane = "xz" if np.allclose(ray_direction, [0, 1, 0]) else "other_plane"
    
    projections = [project_triangle_to_plane(points[tri], plane) for tri in triangles]
    
    # 遍历所有三角形的投影，检查重叠
    for i in range(len(projections)):
        for j in range(i + 1, len(projections)):
            if check_projection_overlap(projections[i], projections[j]):
                overlapping_triangles.append(i)
                break  # 一旦发现重叠就可以停止检测该三角形
    
    return overlapping_triangles


# 检测倒凹区域，只检测那些通过预筛选的三角形
def detect_undercut_with_ray_casting_optimized(points, crown_indices, triangles, ray_direction):
    undercut_points = []
    
    # 预筛选出与其他投影重叠的三角形
    overlapping_triangles = find_overlapping_triangles(points, triangles, ray_direction)
    
    # 遍历牙冠点云中的每个点
    for i, point in enumerate(points[crown_indices]):
        is_undercut = False
        print(f"Checking point {i}/{len(crown_indices)}")
        
        # 仅遍历那些通过预筛选的三角形
        for tri_idx in overlapping_triangles:
            triangle = points[triangles[tri_idx]]
            
            # 使用Möller–Trumbore算法判断射线与三角形相交
            intersect, _ = ray_triangle_intersection(point, ray_direction, triangle)
            
            if intersect:
                is_undercut = True
                break  # 一旦有相交就可以停止检测
        
        # 如果检测到相交，说明当前点是倒凹点
        if is_undercut:
            undercut_points.append(point)
    
    return np.array(undercut_points)


# 可视化倒凹和点云
def visualize_undercut(point_cloud, undercut_points, ray_direction):
    # 创建Open3D点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    
    # 创建用于表示倒凹区域的点云
    undercut_pcd = o3d.geometry.PointCloud()
    undercut_pcd.points = o3d.utility.Vector3dVector(undercut_points)
    
    # 将点云颜色设为浅蓝色，倒凹区域设为红色
    pcd.paint_uniform_color([0.6, 0.8, 1.0])
    undercut_pcd.paint_uniform_color([1.0, 0.0, 0.0])
    
    # 可视化就位道方向
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
    
    # 创建用于表示就位道方向的虚线箭头
    ray_origin = np.array([0, 0, 0])
    ray_end = ray_origin + ray_direction * 10  # 延长射线以便可视化
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector([ray_origin, ray_end]),
        lines=o3d.utility.Vector2iVector([[0, 1]])
    )
    line_set.paint_uniform_color([0, 0, 0])  # 设置为黑色
    
    # 显示点云、倒凹和方向箭头
    o3d.visualization.draw_geometries([pcd, undercut_pcd, line_set, axis])


# 测试代码
if __name__ == "__main__":
    # 1. 加载或生成点云和三角面片数据
    obj_encoding = detect_encoding(obj_file_path)
    obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
    checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces, 5000)  # 对点数小于5000的点云做插值加密
    vertices, labels = load_labeled_point_cloud(txt_file_path)
    crown_idx = [idx for idx in range(len(labels)) if labels[idx]!=1]

    point_cloud = vertices
    triangles = np.array(faces)
    crown_indices = np.array(crown_idx)
    
    # 2. 设置就位道方向
    ray_direction = np.array([0, 1, 0])  # 向上的射线方向

    # 3. 优化后的倒凹区域检测
    undercut_points = detect_undercut_with_ray_casting_optimized(point_cloud, crown_indices, triangles, ray_direction)
    print(f"Detected {len(undercut_points)} undercut points")
    
    # 4. 可视化倒凹和点云
    visualize_undercut(point_cloud, undercut_points, ray_direction)


使用了 1 轮插值来让点云点数满足要求
Checking point 0/5031
Checking point 1/5031
Checking point 2/5031
Checking point 3/5031
Checking point 4/5031
Checking point 5/5031
Checking point 6/5031
Checking point 7/5031
Checking point 8/5031
Checking point 9/5031
Checking point 10/5031
Checking point 11/5031
Checking point 12/5031
Checking point 13/5031
Checking point 14/5031
Checking point 15/5031
Checking point 16/5031
Checking point 17/5031
Checking point 18/5031
Checking point 19/5031
Checking point 20/5031
Checking point 21/5031
Checking point 22/5031
Checking point 23/5031
Checking point 24/5031
Checking point 25/5031
Checking point 26/5031
Checking point 27/5031
Checking point 28/5031
Checking point 29/5031
Checking point 30/5031
Checking point 31/5031
Checking point 32/5031
Checking point 33/5031
Checking point 34/5031
Checking point 35/5031
Checking point 36/5031
Checking point 37/5031
Checking point 38/5031
Checking point 39/5031
Checking point 40/5031
Checking point 41/5031
Checking point 42/5031
C

KeyboardInterrupt: 